## 膜电压计算

$$V(t)=\sum_{i}^{}w_{i} \sum_{t_{i}}^{}K(t-t_{i})+V_{rest}　\tag{1}$$

##  $t_{max}$
$$t_{max} =\frac{ (\tau  * \tau_{s} * log(\frac{\tau}{\tau_{s}}  ))}{ (\tau - \tau_{s})} \tag{2}$$

$$V(t) =\sum_{i}^{}w_{i}\sum_{t_{i}}^{}V_{0}(\exp[-(t-t_{i})/\tau ] - \exp [-(t-t_{i})/\tau_{s}] ) \tag{2.1}$$